# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import io

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

url = "https://en.wikipedia.org/wiki/FIFA_Men%27s_World_Ranking"
response = requests.get(url, headers=headers)


print("Estado:", response.status_code)




Estado: 200


In [8]:
html = io.StringIO(response.text)
tables = pd.read_html(html)

print(f"Se encontraron {len(tables)} tablas.")


Se encontraron 19 tablas.


In [ ]:
df = tables[0] 
df.head()


,0,1,2,3
0,NaN,NaN,NaN,NaN
1,Top 20 rankings as of 19 January 2026[1],Top 20 rankings as of 19 January 2026[1],Top 20 rankings as of 19 January 2026[1],Top 20 rankings as of 19 January 2026[1]
2,Rank,Change,Team,Points
3,1,NaN,Spain,1877.18
4,2,NaN,Argentina,1873.33


In [16]:
df.columns = df.columns.map(str)

df.columns = df.columns.str.strip()

for col in df.columns:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(r"\[.*?\]", "", regex=True)
        .str.strip()
    )

df.replace("", pd.NA, inplace=True)

df.head()



,0,1,2,3
0,NaN,NaN,NaN,NaN
1,Top 20 rankings as of 19 January 2026,Top 20 rankings as of 19 January 2026,Top 20 rankings as of 19 January 2026,Top 20 rankings as of 19 January 2026
2,Rank,Change,Team,Points
3,1,NaN,Spain,1877.18
4,2,NaN,Argentina,1873.33


In [17]:
conn = sqlite3.connect("football_clubs.db")

df.to_sql("football_clubs", conn, if_exists="replace", index=False)

cursor = conn.cursor()

cursor.execute("SELECT COUNT(*) FROM football_clubs")
print("Rows inserted:", cursor.fetchone()[0])

conn.commit()
conn.close()


Rows inserted: 26


In [ ]:
import matplotlib.pyplot as plt

print("Columnas disponibles:")
print(df.columns)

df = df.reset_index(drop=True)
df["Rank"] = df.index + 1

top10 = df.nsmallest(10, "Rank")

top10



Columnas disponibles:
Index(['0', '1', '2', '3'], dtype='str')


,0,1,2,3,Rank
0,NaN,NaN,NaN,NaN,1
1,Top 20 rankings as of 19 January 2026,Top 20 rankings as of 19 January 2026,Top 20 rankings as of 19 January 2026,Top 20 rankings as of 19 January 2026,2
2,Rank,Change,Team,Points,3
3,1,NaN,Spain,1877.18,4
4,2,NaN,Argentina,1873.33,5
5,3,NaN,France,1870,6
6,4,NaN,England,1834.12,7
7,5,NaN,Brazil,1760.46,8
8,6,NaN,Portugal,1760.38,9
9,7,NaN,Netherlands,1756.27,10
